In [1]:
import os

# import
import pandas as pd

In [2]:
# import files 
main_df = pd.read_csv('../PM2.5_Vehicles_Population.csv')
df_pop = pd.read_csv('../California_Population.csv') 

In [3]:
main_df
# main_df['Population Z Score'] = main_df['Population Z Score'].round(2)

,County,Data Year,Total Population,Total Population Z Score,Number of Gas Cars,Number of EV,Population_y,Z Score_y
0,Alameda,2010,1510271,0.033519,885402,20,1510271,0.033519
1,Amador,2010,38091,-0.251576,30480,1,38091,-0.251576
2,Butte,2010,220000,-0.216348,133169,0,220000,-0.216348
3,Calaveras,2010,45578,-0.250126,39636,0,45578,-0.250126
4,Colusa,2010,21419,-0.254805,14610,0,21419,-0.254805
...,...,...,...,...,...,...,...,...
793,Tulare,2023,474680,-0.167028,331744,3837,474680,-0.167028
794,Tuolumne,2023,54626,-0.248374,54369,698,54626,-0.248374
795,Ventura,2023,825960,-0.099001,651851,31980,825960,-0.099001
796,Yolo,2023,220454,-0.216260,152560,6757,220454,-0.216260


In [4]:
# total regestered car users/population 
def create_comparison(county):
    df = main_df[main_df['County'] == county]
    df.loc[:,'Total Cars'] = df['Number of Gas Cars'] + df['Number of EV']
    df.loc[:,'Total Cars/Population'] = (df['Total Cars'] / df['Total Population']).round(2)
    df.loc[:,'EV/Total Cars'] = (df['Number of EV'] / df['Total Cars']).round(2)
    df.loc[:,'Gas/Total Cars'] = (df['Number of Gas Cars'] / df['Total Cars']).round(2)
    return df[['Total Population Z Score','Total Cars/Population', 'EV/Total Cars', 'Gas/Total Cars']] 


In [5]:
# so as more people got cars, more ev were regestered. 
# what is the relationship between population growth and ev adaptation?
## it seems to be positive, direct relationship 
## so as more population increases, ev car adaption also increases. i wonder if this is the same across counties? 

In [6]:
san_diego = create_comparison('San Diego')
san_diego

/var/folders/zc/cqjcs2293rz1ymjq5gj5xxb00000gn/T/ipykernel_48848/4196075774.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'Total Cars'] = df['Number of Gas Cars'] + df['Number of EV']
/var/folders/zc/cqjcs2293rz1ymjq5gj5xxb00000gn/T/ipykernel_48848/4196075774.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'Total Cars/Population'] = (df['Total Cars'] / df['Total Population']).round(2)
/var/folders/zc/cqjcs2293rz1ymjq5gj5xxb00000gn/T/ipykernel_48848/4196075774.py:6: SettingWithCop

,Total Population Z Score,Total Cars/Population,EV/Total Cars,Gas/Total Cars
35,0.340471,0.62,0.00,1.00
92,0.346217,0.61,0.00,1.00
149,0.352168,0.61,0.00,1.00
206,0.358870,0.63,0.00,1.00
263,0.364561,0.64,0.00,1.00
320,0.370074,0.65,0.01,0.99
377,0.372945,0.67,0.01,0.99
434,0.376209,0.75,0.01,0.99
491,0.378975,0.75,0.01,0.99
548,0.380665,0.76,0.02,0.98


In [7]:
 # los_angeles = create_comparison('Los Angeles')
# los_angeles

In [8]:
# orange = create_comparison('Orange')
# orange

In [9]:
# san_fran =  create_comparison('San Francisco')
# san_fran

In [10]:
# mono = create_comparison('Mono')
# mono 

In [11]:
# is this crazy?
# function takes dataset from online and turns it into a merge-able df for main_df 

In [13]:
def aged_population(dataset, datayear):
    # load files 
    df = pd.read_csv(dataset)
    df = df[df['Label (Grouping)'].str.strip() == '18 years and over']
    df = df[df['Label (Grouping)'].str.contains('18 years and over', case=False, na=False)]
    # extract unique county names
    county_names = [col.split("!!")[0] for col in df.columns if "California" in col]
    county_names = list(set(county_names))
    # extract county total and margin of error from main dataset
    county_total = [f"{county}!!Total!!Estimate" for county in county_names]
    county_margin_of_error = [f"{county}!!Total!!Margin of Error" for county in county_names]
    # making df of those
    county_total = df.loc[:, county_total]
    county_margin_of_error = df.loc[:, county_margin_of_error]
    # creating county total population estimate df 
    ct_df =county_total.transpose()
    ct_df = ct_df.reset_index() 
    # some years record >21, others >18. 
    ct_df.columns = ['County' , 'Population Estimate, >18']
    ct_df['County'] = ct_df['County'].str.split(' County').str[0]
    ct_df = ct_df.sort_values(by='County', ascending=True).reset_index(drop=True)
    
    # creating county margin of error 
    me_df = county_margin_of_error.transpose()
    me_df = me_df.reset_index() 
    me_df.columns = ['County' , 'Population Margin of Error, >18']
    me_df['County'] = me_df['County'].str.split(' County').str[0]
    me_df = me_df.sort_values(by='County', ascending=True).reset_index(drop=True) 
    
    # merging on county 
    ct_me_df = pd.merge(ct_df, me_df, how='inner', on = ['County'])
    ct_me_df['Data Year'] = datayear
    return ct_me_df

In [23]:
# run function to create mergable df
totpop2010 = aged_population('./Pop_Over_21_2010.S0101-2024-09-27T210322.csv', 2010)
totpop2011 = aged_population('./Pop_Over_21_2011.S0101-2024-09-27T210305.csv', 2011)
totpop2012 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2012)
totpop2013 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2013)
totpop2014 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2014)
totpop2015 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2015)
totpop2016 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2016)
totpop2017 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2017)
totpop2018 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2018)
totpop2019 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2019)
totpop2020 = aged_population('./pop_over_18_2020.csv', 2020)
totpop2021 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2021)
totpop2022 = aged_population('./Pop_Over_21_2022.csv', 2022)
totpop2023 = aged_population('./Pop_Over_21_2023.S0101-2024-09-27T205616.csv', 2023)
# stacking totpop
dfs = [totpop2010, totpop2011, totpop2012, totpop2013, totpop2014, totpop2015, totpop2016, totpop2017, totpop2018, totpop2019, totpop2021, totpop2022, totpop2023]
aged_pop_2010_2023 = pd.concat(dfs, ignore_index= True)

In [31]:
aged_pop_2010_2023['County'] = aged_pop_2010_2023['County'].str.strip()
counties = pd.DataFrame({
    'County_main_df': main_df['County'],
    'County_aged_pop_2010_2023': aged_pop_2010_2023['County']
})
counties.head()
# Get unique counties from both DataFrames
main_df_co = set(main_df['County'])
aged_pop_2010_2023_co = set(aged_pop_2010_2023['County'])

# Find differences
only_in_main_df_co = main_df_co - aged_pop_2010_2023_co
only_in_aged_pop_2010_2023_co = aged_pop_2010_2023_co - main_df_co

print("Only in main_df:", only_in_main_df_co)
print("Only in aged_pop_2010_2023:", only_in_aged_pop_2010_2023_co)

Only in main_df: set()
Only in aged_pop_2010_2023: {'Alpine'}


In [32]:
# merging aged_pop_2010_2023 to main_df 
aged_pop_df = pd.merge(main_df, aged_pop_2010_2023, how='inner', on = ['Data Year', 'County'])

In [33]:
# there is some percentages and whole numbers, let's make them all whole numbers 
def convert_to_population_estimate(row):
    # converting percentage to decimal, if applicable 
    if isinstance(row['Population Estimate, >18'], str) and '%' in row['Population Estimate, >18']:
        percent_value = float(row['Population Estimate, >18'].strip('%')) / 100
        return percent_value * row['Total Population']
    else:
        return row['Population Estimate, >18']

# apply to update 'Total Population Estimate, >21'
aged_pop_df['Population Estimate, >18'] = aged_pop_df.apply(convert_to_population_estimate, axis=1)



aged_pop_df = aged_pop_df[['County', 'Data Year', 'Total Population', 'Total Population Z Score', 
                           'Population Estimate, >18', 'Population Margin of Error, >18', 
                           'Number of Gas Cars', 'Number of EV'
                           ]]

aged_pop_df

,County,Data Year,Total Population,Total Population Z Score,"Population Estimate, >18","Population Margin of Error, >18",Number of Gas Cars,Number of EV
0,Alameda,2010,1510271,0.033519,1170460.025,*****,885402,20
1,Butte,2010,220000,-0.216348,174460.0,±0.3,133169,0
2,Contra Costa,2010,1049025,-0.055803,788866.8,±0.1,666294,10
3,El Dorado,2010,181058,-0.223890,139776.776,±0.2,129980,0
4,Fresno,2010,930450,-0.078766,652245.45,±0.1,481145,2
...,...,...,...,...,...,...,...,...
534,Tehama,2023,64710,-0.246421,"49,287",*****,48496,398
535,Tulare,2023,474680,-0.167028,"338,510",±144,331744,3837
536,Ventura,2023,825960,-0.099001,"651,462",*****,651851,31980
537,Yolo,2023,220454,-0.216260,"178,433",±709,152560,6757


In [27]:
# to csv
aged_pop_df.to_csv('Main_DF_PM2.5_Vehicle_Pop.csv', index=False)